In [2]:
import papermill as pm
from pathlib import Path

In [3]:
import psutil

# Get memory information
memoryInfo = psutil.virtual_memory()
total_memory = memoryInfo.total / (1024 ** 3)
available_memory = memoryInfo.available / (1024 ** 3)
used_memory = memoryInfo.used / (1024 ** 3)

print(f"Total Memory: {total_memory:.2f} GB")
print(f"Available Memory: {available_memory:.2f} GB")
print(f"Used Memory: {used_memory:.2f} GB")

Total Memory: 125.58 GB
Available Memory: 98.95 GB
Used Memory: 25.38 GB


# 0 - Run `generate_nrrd_volumes` function

In [4]:
import shutil


def _run_tif2nrrd_notebook(
    sample_dir,
    notebook_name,
    tiff_subdir=None,
):
    """
    Internal helper to execute a tif2nrrd notebook using the default
    slices_<sample_name> structure unless overridden.
    """
    sample_dir_path = Path(sample_dir)
    sample_name = sample_dir_path.stem

    if tiff_subdir is None:
        tiff_subdir = f"slices_{sample_name}"

    tiff_path = sample_dir_path / tiff_subdir

    paper_dict = dict(
        image_directory_name=str(tiff_path),
        output_dir_path=str(sample_dir_path / "microCT_volume"),
        pca_directory_name=str(sample_dir_path),
    )

    input_nb = notebook_name
    output_nb = str(sample_dir_path / notebook_name)

    print(input_nb)
    pm.execute_notebook(
        input_nb,
        output_nb,
        parameters=paper_dict,
    )


def generate_nrrd_volume_preview(
    sample_dir,
    tiff_subdir=None,
):
    """
    Generate preview quality NRRD volumes.
    """
    _run_tif2nrrd_notebook(
        sample_dir=sample_dir,
        notebook_name="00_tif2nrrd_previewQuality.ipynb",
        tiff_subdir=tiff_subdir,
    )


def generate_nrrd_volume_full(
    sample_dir,
    tiff_subdir=None,
    delete_tiff=False,
):
    """
    Generate full quality NRRD volumes and optionally delete TIFF files.
    """
    sample_dir_path = Path(sample_dir)
    sample_name = sample_dir_path.stem

    if tiff_subdir is None:
        tiff_subdir = f"slices_{sample_name}"

    tiff_path = sample_dir_path / tiff_subdir

    _run_tif2nrrd_notebook(
        sample_dir=sample_dir,
        notebook_name="00_tif2nrrd_fullQuality.ipynb",
        tiff_subdir=tiff_subdir,
    )

    if delete_tiff:
        try:
            shutil.rmtree(tiff_path)
            print(f"Deleted TIFF directory: {tiff_path}")
        except Exception as e:
            print(f"Error deleting TIFF directory {tiff_path}: {e}")


def generate_nrrd_volumes(
    sample_dir,
    tiff_subdir=None,
    delete_tiff=False,
):
    """
    Run preview and full quality generation using the standard sample layout.
    """
    generate_nrrd_volume_preview(
        sample_dir,
        tiff_subdir=tiff_subdir,
    )

    generate_nrrd_volume_full(
        sample_dir,
        tiff_subdir=tiff_subdir,
        delete_tiff=delete_tiff,
    )


In [11]:
sample_dir = '/config/researcher_home/Documents/microCT/2025-12-17_GN009/104574/'

# generate_nrrd_volume_preview(sample_dir)
# generate_nrrd_volume_full(sample_dir, delete_tiff=True)
generate_nrrd_volumes(sample_dir, delete_tiff=True)

00_tif2nrrd_previewQuality.ipynb


Executing: 100%|##########| 27/27 [00:18<00:00,  1.45cell/s]


00_tif2nrrd_fullQuality.ipynb


Executing: 100%|##########| 29/29 [06:09<00:00, 12.74s/cell]


Deleted TIFF directory: /config/researcher_home/Documents/microCT/2025-12-17_GN009/104574/slices_104574


# 1 - Run `thresholdInspector` function

In [5]:
def thresholdInspector(sample_dir, volume_file = 'microCT_volume/microCT_volume.nrrd'):
    
    sample_path = Path(sample_dir)

    volume_file_path = sample_path / volume_file
    output_dir = sample_path / 'segmented_volumes'

    paper_dict = dict(volume_file=str(volume_file_path),
                  output_dir_path=str(output_dir))

    input_nb = '01_inspect_thresholds.ipynb'
    output_nb =  sample_path / input_nb

    print(input_nb)
    pm.execute_notebook(
       input_nb,
       str(output_nb),
       parameters=dict(paper_dict)
    );

In [13]:
sample_dir = '/config/researcher_home/Documents/microCT/2025-12-17_GN009/104574/'

# thresholdInspector(sample_dir, volume_file = 'microCT_volume/microCT_volume_preview.nrrd')
thresholdInspector(sample_dir)

01_inspect_thresholds.ipynb


Executing: 100%|##########| 48/48 [11:05<00:00, 13.87s/cell]


# Run `microCT_analyzer` function

In [8]:
def microCT_analyzer(sample_dir, volume_file = 'microCT_volume/microCT_volume.nrrd'):  

    sample_path = Path(sample_dir)

    volume_file_path = sample_path / volume_file
    output_dir = sample_path / 'segmented_volumes'
    segmentMask_file_path = output_dir / 'ROI.seg.nrrd'
    segments_greyvalues_file_path = output_dir / 'segments_greyvalues.csv'
    
    # 02_segment_microCT
    paper_dict = dict(volume_file = str(volume_file_path),
                  output_dir_path = str(output_dir),
                  segmentMask_file = str(segmentMask_file_path),
                  segments_greyvalues_file = str(segments_greyvalues_file_path)
                 )

    input_nb = '02_segment_microCT.ipynb'
    output_nb =  sample_dir + input_nb

    print(input_nb)
    pm.execute_notebook(
       input_nb,
       output_nb,
       parameters=dict(paper_dict)
    );

In [10]:
sample_dir = '/config/researcher_home/Documents/microCT/2025-12-17_GN009/104574/'

microCT_analyzer(sample_dir, volume_file = 'microCT_volume/microCT_volume_preview.nrrd')

02_segment_microCT.ipynb


Executing: 100%|##########| 87/87 [02:34<00:00,  1.78s/cell]
